<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Исследование-баланса-классов" data-toc-modified-id="Исследование-баланса-классов-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Исследование баланса классов</a></span></li></ul></li><li><span><a href="#Лемматизация-и-чистка-текста" data-toc-modified-id="Лемматизация-и-чистка-текста-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация и чистка текста</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка признаков</a></span></li><li><span><a href="#Векторизация-признаков" data-toc-modified-id="Векторизация-признаков-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Векторизация признаков</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import optuna

from pymystem3 import Mystem
import re
from nltk.corpus import stopwords as nltk_stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')


In [3]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [5]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
data.drop('Unnamed: 0', axis=1, inplace=True)

#### Исследование баланса классов

In [8]:
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

<b>Вывод:</b> В Данных наблюдается дисбаланс классов. При обучение моделей необходимо вводить параметр class_weight='balanced' и использовать метрику f1 

### Лемматизация и чистка текста

In [9]:
m = WordNetLemmatizer()

def lemmatize_clear(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
    text = ' '.join(text.split())
    text = nltk.word_tokenize(text)
    return ' '.join([m.lemmatize(word, wordnet.VERB) 
            for word, tag in pos_tag(text)])

data['lemm_text'] = data['text'].apply(lemmatize_clear)
data = data.drop(['text'], axis=1)
del m

In [10]:
data.head()

,toxic,lemm_text
0,0,explanation why the edit make under my usernam...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not try to edit war it s ju...
3,0,more i can t make any real suggestions on impr...
4,0,you sir be my hero any chance you remember wha...


### Подготовка признаков

In [11]:
RANDOM_STATE = 12345

In [12]:
features = data['lemm_text']
target = data['toxic']

In [13]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.25, 
                                                                            random_state=RANDOM_STATE,
                                                                            stratify=target)

### Векторизация признаков

In [14]:
nltk.download('stopwords')
stopwords = list(set(nltk_stopwords.words('english')))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Обучение

### Логистическая регрессия

In [15]:
features_train_lr, target_train_lr = features_train.copy(), target_train.copy()

pipeline = Pipeline(
    [
        ('vect', TfidfVectorizer(stop_words=stopwords)),
        ('modellr', LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced', solver='liblinear')),
    ]
)
parametr = {'modellr__penalty' : ['l1','l2'],
            'modellr__C' : list(range(5, 15)),
            }


random_search_lr = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=parametr,
    n_iter=20,
    random_state=0,
    n_jobs=-1,
    scoring='f1',
    verbose=1
)
random_search_lr.fit(features_train_lr, target_train_lr)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(estimator=Pipeline(steps=[('vect',
                                              TfidfVectorizer(stop_words=["it's",
                                                                          'isn',
                                                                          'too',
                                                                          'themselves',
                                                                          'very',
                                                                          "won't",
                                                                          "mustn't",
                                                                          'doing',
                                                                          'couldn',
                                                                          'we',
                                                                          'below',
                                  

In [16]:
model_lr = random_search_lr.best_estimator_

### Случайный лес

In [17]:
features_train_rf, target_train_rf = features_train.copy(), target_train.copy()

In [18]:
def rf_objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 10, 40)
    max_depth = trial.suggest_int('max_depth', 10,40)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 1, 10) 
    pipeline = Pipeline(
    [
        ('vect', TfidfVectorizer(stop_words=stopwords)),
        ('modellr', RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators,
                                           min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, 
                                           random_state=RANDOM_STATE, class_weight='balanced')),
    ]
)
    scores = cross_val_score(pipeline, features_train_rf, target_train_rf, scoring='f1')
    
    return scores.mean()
    
rf_study = optuna.create_study(direction='maximize')
rf_study.optimize(rf_objective, n_trials=20)

[I 2023-04-07 07:59:47,587] A new study created in memory with name: no-name-27f784eb-8895-4c7e-9029-2427a5581167
[W 2023-04-07 08:00:12,763] Trial 0 failed with parameters: {'n_estimators': 26, 'max_depth': 37, 'min_samples_leaf': 6, 'min_samples_split': 1} because of the following error: The value nan is not acceptable..
[W 2023-04-07 08:00:12,764] Trial 0 failed with value nan.
[I 2023-04-07 08:00:59,627] Trial 1 finished with value: 0.41963905630151965 and parameters: {'n_estimators': 35, 'max_depth': 36, 'min_samples_leaf': 6, 'min_samples_split': 7}. Best is trial 1 with value: 0.41963905630151965.
[W 2023-04-07 08:01:24,309] Trial 2 failed with parameters: {'n_estimators': 17, 'max_depth': 23, 'min_samples_leaf': 9, 'min_samples_split': 1} because of the following error: The value nan is not acceptable..
[W 2023-04-07 08:01:24,310] Trial 2 failed with value nan.
[I 2023-04-07 08:02:05,096] Trial 3 finished with value: 0.3732780219417381 and parameters: {'n_estimators': 34, 'max_

### CatBoost

In [19]:
text_features = ['lemm_text']
target = ['toxic']

In [20]:
features_train_cb, target_train_cb = features_train.copy(), target_train.copy()
catboost = CatBoostClassifier(iterations=100, learning_rate=0.1, text_features=text_features, verbose=3)
cv_cb= cross_validate(catboost, pd.DataFrame(features_train)[text_features], pd.DataFrame(target_train)[target], scoring='f1')

0:	learn: 0.6159396	total: 553ms	remaining: 54.8s
3:	learn: 0.4564500	total: 2.04s	remaining: 48.9s
6:	learn: 0.3575629	total: 3.56s	remaining: 47.4s
9:	learn: 0.2927847	total: 5.05s	remaining: 45.4s
12:	learn: 0.2480202	total: 6.62s	remaining: 44.3s
15:	learn: 0.2176676	total: 8.15s	remaining: 42.8s
18:	learn: 0.1962040	total: 9.68s	remaining: 41.3s
21:	learn: 0.1809509	total: 11.3s	remaining: 39.9s
24:	learn: 0.1700712	total: 12.8s	remaining: 38.5s
27:	learn: 0.1623313	total: 14.4s	remaining: 36.9s
30:	learn: 0.1563113	total: 15.9s	remaining: 35.3s
33:	learn: 0.1522182	total: 17.4s	remaining: 33.7s
36:	learn: 0.1492489	total: 18.8s	remaining: 32.1s
39:	learn: 0.1460756	total: 20.5s	remaining: 30.7s
42:	learn: 0.1443691	total: 21.9s	remaining: 29s
45:	learn: 0.1423913	total: 23.4s	remaining: 27.4s
48:	learn: 0.1409398	total: 24.8s	remaining: 25.9s
51:	learn: 0.1398265	total: 26.4s	remaining: 24.3s
54:	learn: 0.1385840	total: 27.9s	remaining: 22.9s
57:	learn: 0.1375166	total: 29.5s	rem

In [21]:
pd.DataFrame({'Качество модели RMSE':[random_search_lr.best_score_,rf_study.best_value,cv_cb['test_score'].mean()]},
            index=['Логистическая регрессия', 'Случайный лес', 'CatBoost'])

,Качество модели RMSE
Логистическая регрессия,0.764534
Случайный лес,0.429311
CatBoost,0.763531


## Выводы

In [22]:
f1_score(target_test, model_lr.predict(features_test))

0.7676975945017183

In [23]:
cb = CatBoostClassifier(iterations=100, learning_rate=0.1, text_features=text_features, verbose=3).fit(pd.DataFrame(features_train)[text_features], pd.DataFrame(target_train)[target])
f1_score(target_test, cb.predict(pd.DataFrame(features_test)[text_features]))

0:	learn: 0.6148155	total: 547ms	remaining: 54.2s
3:	learn: 0.4535447	total: 2.22s	remaining: 53.2s
6:	learn: 0.3556785	total: 3.91s	remaining: 51.9s
9:	learn: 0.2899937	total: 5.61s	remaining: 50.5s
12:	learn: 0.2456422	total: 7.37s	remaining: 49.3s
15:	learn: 0.2146704	total: 9.05s	remaining: 47.5s
18:	learn: 0.1931291	total: 10.8s	remaining: 46s
21:	learn: 0.1781687	total: 12.5s	remaining: 44.2s
24:	learn: 0.1672260	total: 14.2s	remaining: 42.6s
27:	learn: 0.1593998	total: 15.8s	remaining: 40.8s
30:	learn: 0.1534622	total: 17.6s	remaining: 39.3s
33:	learn: 0.1498465	total: 19.3s	remaining: 37.5s
36:	learn: 0.1466329	total: 21s	remaining: 35.8s
39:	learn: 0.1438729	total: 22.7s	remaining: 34.1s
42:	learn: 0.1420788	total: 24.4s	remaining: 32.3s
45:	learn: 0.1405050	total: 26s	remaining: 30.5s
48:	learn: 0.1388392	total: 27.8s	remaining: 28.9s
51:	learn: 0.1376610	total: 29.5s	remaining: 27.2s
54:	learn: 0.1368031	total: 31.1s	remaining: 25.5s
57:	learn: 0.1354899	total: 33s	remaining

0.7624965479149407

<b>Выводы</b>:
   - В данной работе проведена обработка тескта(леммитилизация и чистка текста)
   - Выбран способ балансирвоания классов 
   - Исходные данные обладают большим количеством признаков. Обучены модели Логистическая регрессия, Случайный лес, CatBoost. На основе кросс валидации лучшей моделью оказались Логистическая регрессия и catboost с показателем метрbrb f1=0.76 . у модель Случайный лес F1=0.44

На тестовой выборке метрике F1 равна 0.76 как для Логистической Регресии , так и для catboost